# Train loop 

In [ ]:
import torch
import cv2
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt 
from glob import glob
from PIL import Image

In [ ]:
class CV_Dataset:
  def __init__(self, images_paths, targets, augmentations=None):
    self.img_path = images_paths
    self.targets = targets
    self.augmentations = augmentations
  
  def __len__(self):
    return len(self.img_path)
  
  def __getitem__(self, idx):
    image = cv2.imread(self.img_path[idx])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # some image formatting before passing to tensor
    if self.augmentations is not None:
      img_augmented = self.augmentations(image=image)
      image = img_augmented["image"]
    if self.targets.shape[0] > 1:
      labels = self.targets.iloc[idx]      
    else:
      labels = self.targets[idx]
    sample = {'image': torch.tensor(image), 
              'targets': torch.tensor(labels)}
    return sample


In [ ]:
!mkdir data
num_images = 1000
path_to_save_imgs = "/content/data"

for i in range(num_images):
  random_image_np = np.random.randn(32,32,3).astype(np.uint8)
  random_image_pil = Image.fromarray(random_image_np)
  random_image_pil.save(os.path.join(path_to_save_imgs,f"{i}.jpg"))

mkdir: cannot create directory ‘data’: File exists


In [ ]:
all_imgs_paths = sorted(glob(f"{path_to_save_imgs}/*.jpg"))
labels = [np.random.randint(0,10,len(all_imgs_paths)).astype(np.uint8), np.random.randint(0,10,len(all_imgs_paths)).astype(np.uint8)]
data = zip(all_imgs_paths,np.array(labels).T.tolist())
df = pd.DataFrame(data=data, columns=["imgs_path","labels"])
custom_dataset = CV_Dataset(df.imgs_path, df.labels)


In [ ]:
class MSE:
  def __call__(self, pred, target):
    self.target = target
    self.pred = pred
    return ((target - pred)** 2).mean()

  def backward(self):
    n = self.target.shape[0]
    self.gradient = 2. * (self.pred - self.target) / n
    return self.gradient


In [ ]:
class Linear:
  def __init__(self, input_dim, num_hidden=1):
    self.weights =  torch.tensor(np.random.randn(input_dim, num_hidden) * np.sqrt(2. / input_dim), dtype = torch.double)
    self.bias = torch.tensor(np.zeros(num_hidden), dtype = torch.double)

  def __call__(self,x):
    self.x = x
    return x @ self.weights + self.bias

  def backward(self, gradient):
    self.weights_gradient = self.x.T @ gradient
    self.bias_gradient =  gradient.sum(axis=0)    
    self.x_gradient = gradient @ self.weights.T
    return self.x_gradient

  def update(self, lr):   
    self.weights = self.weights - self.weights_gradient* lr
    self.bias = self.bias - lr * self.bias_gradient



In [ ]:
class ReLU:
  def __call__(self, input_):
    self.input_  = input_
    self.output = np.clip(self.input_,0, None)
    return self.output

  def backward(self, output_gradient):
    self.input_gradient = (self.input_ > 0) * output_gradient
    return self.input_gradient



In [ ]:
class Model:
  def __init__(self,input_dim, num_hidden, num_classes=1):
    self.linear1 = Linear(input_dim, num_hidden)
    self.relu = ReLU()
    self.linear2 = Linear(num_hidden, num_classes)

  def __call__(self,x):
    l1 = self.linear1(x)
    r = self.relu(l1)
    l2 = self.linear2(r)
    return l2

  def backward(self, output_gradient):
    linear2_gradient = self.linear2.backward(output_gradient)
    relu_gradient = self.relu.backward(linear2_gradient)
    linear1_gradient = self.linear1.backward(relu_gradient)
    return linear1_gradient


  def update(self, lr):
    self.linear2.update(lr)
    self.linear1.update(lr)



    



In [ ]:
train_data, val_data = torch.utils.data.random_split(custom_dataset, [750, 250])
train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=4, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_data, shuffle=True, batch_size=4, num_workers=2)

In [ ]:
model = Model(3*32*32, 1, 1)
loss = MSE()

In [ ]:
def torch_fit(loader, model, loss, lr, num_target=0, num_epochs=1):
  for epoch in range(num_epochs):
    for data in train_loader:
       train_im =  torch.tensor(data['image'].view(-1, 3*32*32), dtype = torch.double)
       train_tar = torch.tensor(data['targets'][:, num_target:num_target+1], dtype = torch.double)
       y_pred_tensor = model(train_im)
       loss_value = loss(y_pred_tensor, train_tar)
       print(f'Epoch {epoch} --- loss: {loss_value}')
       gradient_from_loss = loss.backward()
       model.backward(gradient_from_loss)
       model.update(lr)
       break
for i in range(len(custom_dataset[0]['targets'])):
  print(f'---Target {i}---')
  torch_fit(train_loader, model=model, loss=loss, lr=0.1, num_epochs=10)

---Target 0---
Epoch 0 --- loss: 55.083781726189144


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


Epoch 1 --- loss: 32.00438274425259
Epoch 2 --- loss: 12.930015815780614
Epoch 3 --- loss: 18.3786205595772
Epoch 4 --- loss: 9.854360458165232
Epoch 5 --- loss: 2.9871368799306257
Epoch 6 --- loss: 15.624990653791702
Epoch 7 --- loss: 7.967494018426687
Epoch 8 --- loss: 8.973301080149295
Epoch 9 --- loss: 6.142238966536258
---Target 1---
Epoch 0 --- loss: 8.218532938583204
Epoch 1 --- loss: 11.216410551460815
Epoch 2 --- loss: 9.619238440937089
Epoch 3 --- loss: 2.8353803058295726
Epoch 4 --- loss: 7.060964229601803
Epoch 5 --- loss: 5.002903772751752
Epoch 6 --- loss: 15.131186412496959
Epoch 7 --- loss: 6.156715570939941
Epoch 8 --- loss: 10.307797965401562
Epoch 9 --- loss: 14.204418719542616
